SyntaxError: invalid syntax (<ipython-input-5-865c7179a5cf>, line 2)

In [ ]:
import os
import re
import glob 
import matplotlib.pyplot as plt
import numpy as np
from os.path import basename
import audiosegment
from multiprocessing import Pool
modulePath = 'ChristiansPythonLibrary/src' 
import sys
import numpy
sys.path.append(modulePath)
import generalUtility
import dspUtil
import matplotlibUtil
import librosa
import pickle



#Constant
EMOTION_ANNOTATORS = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}

METHOD = {'audio_feature':0, 'LSTM':1, 'LSTM_KERAS':2}

#Method for classification
method = METHOD['LSTM_KERAS']

#If data is processed and saved into files, just reload, dont need to re-process
isRawDataProcessed = True

#Development mode. Only run with small data.
dev = False



#Define class
class Input:
    ##spectral, prosody, erergy are dict type
    def __init__(self, spectral, prosody, energy, spectrogram):
        self.spectral = spectral
        self.prosody = prosody
        self.energy = energy
        self.spectrogram = spectrogram
        
    def print(self):
        print("spectral  features: ", spectral)
        print("prosody features: ", prosody)
        print("energy: ", energy)
        print("spectrogram: ", spectrogram)
        
    def input2Vec(self, onlySpectrogram):
        if (onlySpectrogram ==  False):
            features = []
            s = list(self.spectral.values())
            p = list(self.prosody.values())
            e = list(self.energy.values())
            [features.extend(x) for x in [s, p, e]]
            return features
        else :
            return self.spectrogram
    
class Output:
    def __init__(self, duration, code, category_origin, category_evaluation, attribute):
        self.duration = duration
        self.code = code
        self.category_origin = category_origin
        self.category_evaluation = category_evaluation
        self.attribute = attribute
        
     
    def print(self):
        print("duration: ", self.duration)
        print("code: ", self.code)
        print("category_origin: ", self.category_origin)
        print("category_evaluation: ", self.category_evaluation)
        print("attribute: ", self.attribute)
        
    def output2Vec(self):
        emotion = EMOTION[self.category_origin]
        return emotion
    
    
    
#Functions for get features from audio file
def amp2Db(samples):
    dbs = []
    for  x in samples:
        if x < 0:
            v = - dspUtil.rmsToDb(np.abs(x))
        elif x == 0:
            v = 0
        else :
            v = dspUtil.rmsToDb(np.abs(x))
        dbs.append(v)
    return dbs

def getF0Features(file):
    features = {}
    sound = audiosegment.from_file(file)
    voiced = sound.filter_silence(duration_s=0.2)
    frame_rate = sound.frame_rate
    frames = sound.dice(0.032)

    f0s = []
    for f in frames:
        f0 = dspUtil.calculateF0once(amp2Db(f.get_array_of_samples()), frame_rate)
        if(f0 != 0):
            f0s.append(f0)
    
    features['f0_min'] = np.min(f0s)
    features['f0_max'] = np.max(f0s)
    features['f0_range'] = np.max(f0s) - np.min(f0s)
    features['f0_mean'] = np.mean(f0s)
    features['f0_median'] = np.median(f0s)
    features['f0_25th'] = np.percentile(f0s, 25)
    features['f0_75th'] = np.percentile(f0s, 75)
    features['f0_std'] = np.std(f0s)
    
  
    return features

def getEnergyFeatures(file):
    features = {}
    sound = audiosegment.from_file(file)
    voiced = sound.filter_silence(duration_s=0.2)
    samples = voiced.get_array_of_samples()
    frame_rate = sound.frame_rate
    frames = sound.dice(0.032)
    
    e = []
    for f in frames:
        e.append(np.abs(f.max_dBFS))
    
    
    features['energy_min'] = np.min(e)
    features['energy_max'] = np.max(e)
    features['energy_range'] = np.max(e) - np.min(e)
    features['energy_mean'] = np.mean(e)
    features['energy_median'] = np.median(e)
    features['energy_25th'] = np.percentile(e, 25)
    features['energy_75th'] = np.percentile(e, 75)
    features['energy_std'] = np.std(e)   

    return features
    
def audio2Features(file):
    spectral = {}
    prosody = {}
    energy = {}
    try:
        prosody = getF0Features(file)
        energy = getEnergyFeatures(file)
        
        y, sr = librosa.load(file)
        spectrogram = librosa.stft(y)
        spectrogram = np.abs(spectrogram)
        #To be continued....
    
        return Input(spectral, prosody, energy, spectrogram)
    except Exception as e:
        print(e)
        
        
#Function for getting input vector and corresponding output      
def parallel_task(d0, d1):
    print("task...")
    # Each input diectory contains many file
    # This fucntion will walk through all valid 'wav'files in this directory and get features like engergy, frequency...
    def parseInput(dir):
        dicts = {} 
        for f in os.listdir(dir):
            if not f.startswith(".") and os.path.splitext(f)[1] == ".wav":
                dicts[os.path.splitext(f)[0]] = audio2Features(dir + "/" + f)


        return dicts
    
    # Each output file contains label of many diffrent 'wav' file.
    # This function will parse content of text file using 'regrex'. Then turn it into label
    def parseOutput(file):
        dict_namefile_output = {}
        # Open file to get all contents excepts the first line.
        f = open(file, 'r')
        content = ""
        index = 0
        for line in f:
            index = index + 1
            if index == 1:
                continue
            content  = content + line

        # Find all matched patterns in the content
        ps = re.findall(r'\[.*?\)\n\n', content, re.DOTALL)

        # Parse each matched pattern into  'Output' object
        try:
            for p in ps:
                ls = p.split("\n")
                ls = list(filter(lambda x: len(x) > 0 ,ls))

                # Split elements of the first line which looks like : 
                # [147.0300 - 151.7101]	Ses01F_impro02_M012	neu	[2.5000, 2.0000, 2.0000]
                ele_line0 = re.search(r'(\[.*?\])(\s)(.*?)(\s)(.*?)(\s)(\[.*?\])', ls[0]).groups()

                # Split time components which looks like:
                # [147.0300 - 151.7101]
                time_dur = ele_line0[0]
                ele_time_dur = re.findall(r"[-+]?\d*\.\d+|\d+", time_dur)
                ele_time_dur = [float(x) for x in ele_time_dur]

                # Get code and category_origin which looks like:
                # Code: Ses01F_impro02_M012
                # Category_origin: neu
                code = ele_line0[2]
                category_origin = ele_line0[4]

                # Split attribute components which looks like:
                # [2.5000, 2.0000, 2.0000]
                attribute = ele_line0[6]
                ele_attribute = re.findall(r"[-+]?\d*\.\d+|\d+", attribute)
                ele_attribute = [float(x) for x in ele_attribute]

                # Get categorial_evaluation:
                lines_categorical = list(filter(lambda x : x[0] == 'C', ls))
                rex = re.compile(r'C.*?:(\s)(.*?)(\s)\(.*?\)')

                category_evaluation = []
                for l in lines_categorical:
                    elements = rex.search(l).groups()
                    cat = elements[1]
                    cat = cat.split(";")
                    cat = map(lambda x: x.lstrip(), cat)
                    cat = list(filter(lambda x: len(x)>0, cat))
                    category_evaluation.extend(cat)


                # Make list distinct
                category_evaluation = list(set(category_evaluation))
                
                

                # Make dict {name_file : parsed_output}
                dict_namefile_output[code] = Output(ele_time_dur, code, category_origin, category_evaluation, ele_attribute)
            return dict_namefile_output
        except Exception as e:
            print(e)


    ### Parse input and output files and get input and output as vector
    dicts_in = parseInput(d0)
    dicts_out = parseOutput(d1)
    in_out = []
    
    keys = list(dicts_in.keys())
    for key in keys:
        if(dicts_out[key].category_origin != 'xxx'):
            if (method == METHOD['LSTM']):
                in_out.append((dicts_in[key].input2Vec(onlySpectrogram=True), dicts_out[key].output2Vec()))
            else:
                in_out.append((dicts_in[key].input2Vec(onlySpectrogram=False), dicts_out[key].output2Vec()))
    return in_out
    
    
def createInput_Output():
    ### Get directories of input and output
    DATA_DIR = "IEMOCAP_full_release"
    NUM_SESSION = 5
    input_output = []
    for i in range (1, NUM_SESSION + 1):
        name_session = "Session" + str(i)
        root_dir_of_wav = DATA_DIR + "/" + name_session + "/sentences" + "/wav"
        root_dir_of_labels = DATA_DIR + "/" + name_session + "/dialog" + "/EmoEvaluation"

        for x in os.walk(root_dir_of_wav):
            if(x[0] == root_dir_of_wav):
                dirs_of_wav = x[1]
                index = -1
            else:
                index = index + 1
                input_output.append((x[0], root_dir_of_labels + "/" + dirs_of_wav[index] + ".txt"))
                
    
    ds = input_output
    in_out = []
    input = []
    out = []
    
    # Multi processing
    with Pool(processes=8) as pool:
         in_out = pool.starmap(parallel_task, ds)
   
    r = []
    for e in in_out:
        r = r + e
    
    input = [x[0] for x in r]
    out = [x[1] for x in r]
    print("Finished creating input output into txt file")
    return (input, out)
 


#If have not processed data yet then process, otherwise loading data from file.
if isRawDataProcessed == False:

    ##Get input, normalize input, get output
    input, output = createInput_Output()
    output = np.array(output)
    
    if(method == METHOD['audio_feature']):
        input = np.array(input)
        input = input / input.max(axis=0)
        filehandlerInput = open('processed-data/input0.obj', 'wb')
        filehandlerOutput = open('processed-data/output0.obj', 'wb')
    elif(method == METHOD['LSTM']):
        # After this operator, each sample will be a 2-D array, Each row includes magnitude energy values in range of frquencies
        # Rows will have the same length in all samples.
        # Each sample will have different number of rows beacause their difference of length in seconds 
        #input = [list(map(list, zip(*i))) for i in input]
        filehandlerInput = open('processed-data/input1.obj', 'wb')
        filehandlerOutput = open('processed-data/output1.obj', 'wb')

        
    pickle.dump(input, filehandlerInput)
    pickle.dump(output, filehandlerOutput)
    print("Finish write processed data (input, output) to file!!!")
    
    

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.metrics import confusion_matrix
import random
import timeit
from sklearn.utils import class_weight
start = timeit.default_timer()



dev = True
size_batch = 10
window_size = 25
num_class = 3
if method == METHOD['LSTM_KERAS']: 
    ##Loading  data from files
    filehandlerInput = open('processed-data/input1.obj', 'rb')
    filehandlerOutput = open('processed-data/output1.obj', 'rb')
    input = pickle.load(filehandlerInput)
    output = pickle.load(filehandlerOutput)
    
    #if in development mode, just use the small data!
    if (dev):
        data_size = 300
    else:
        data_size = len(input)
    
    print("\nThe number of samples ", data_size)
    
    # Input : (size of samples, ?, f_range = 1025)
    input = [list(map(list, zip(*i))) for i in input[0:data_size]]
    

    #Normalize input
    max_val = -1
    for i in input:
        b = [max(x) for x in i]
        c = max(b)
        if c >  max_val:
            max_val = c

    print("\nThis is max_value of input: ", max_val)
    
    #Normalize
    for i in range(0, len(input)):
        for j in range(0, len(input[i])):
            input[i][j] = [x / max_val for x in input[i][j]]
            
    
    print("\nFinished normalize input.")

    #Widen the width of the data with the window_size
    for j in range(0,len(input)):
        t = []
        for i in range(0, len(input[j]), window_size):
            flat_list = [item for sublist in input[j][i:i+window_size] for item in sublist]
            t.append(flat_list)
        input[j] = t
        input[j] = t[0:len(t) - 1]
        
    print("\nFinished widen window_size!")
    
    
    output = output[0:data_size]
    
    #Cluster output
    for i in range (0, len(output)):
        if (output[i] % 2 == 0):
            output[i] = 0
        elif (output[i] == 1 or output[i] == 5):
            output[i] = -1
        elif (output[i] == 3):
            output[i] = -2
    
    output = [x * -1 for x in output]
    print("Finish clustering emotion")
    
    # Get original quantiry of each label
    y = np.bincount(output)
    ii = np.nonzero(y)[0]
    a = list(zip(ii,y[ii]))
    #Get quantity of the largest sample.
    max_sample = max(a, key=lambda x:x[1])[1]
    #print("max_sample: ",max_sample)
    
    print("EMOTION_ANNOTATE: ", EMOTION_ANNOTATORS)
    print("\nThe quantity of each label: ", a, "\n")
    
    #Get the class_weight of each class
    
    class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(output)
                                               ,output)
    
    #Remove labels that have small quantity.
    indices = [] 
    for i in range(0, len(output)):
        if output[i] >= 3:
            indices.append(i)
    input = np.delete(input, indices, axis = 0)
    output = np.delete(output, indices)
    print("\nRemoved samples that have the tiny quantity!")

    #shuffle data
    c = list(zip(input, output))
    random.shuffle(c)
    input, output = zip(*c)
    
    print("\nFinished shuffling all data!")
    
    #Group the same values into each bucket
    c = list(zip(input, output))
    values = list(set(map(lambda x:x[1], c)))
    input_groups = [[y[0] for y in c if y[1]==x] for x in values]
  

    # Split train, test
    trainlen = int(0.8 * data_size)
    train_in = []
    train_out = []
    test_in = [] 
    test_out = []
   
    for i in range(0, len(values)):
        out_label = np.zeros((1,num_class))[0]
        out_label[values[i]] = 1

        len_group = len(input_groups[i])
        ratio_sample_int = int(max_sample / len_group)
        ratio_sample_remain = max_sample / len_group - ratio_sample_int
        
        new_train_in = input_groups[i][0:int(0.8 * len_group)]
        
        new_train_in_up_sampling = list(np.repeat(new_train_in, ratio_sample_int, axis = 0))
        number_addition_sample = int(ratio_sample_remain * len(new_train_in)) - len(new_train_in)
        new_train_in_up_sampling = new_train_in_up_sampling + new_train_in[0:number_addition_sample]
       
        train_in  = train_in + new_train_in_up_sampling
        
        print("Up sampling sample ", i, "in training set. ", "The new length is: ", len(new_train_in_up_sampling))
        new_test_in = input_groups[i][int(0.8 * len_group) : len_group]
        test_in = test_in + new_test_in
        train_out = train_out + [out_label] * len(new_train_in_up_sampling)
        test_out = test_out + [out_label] * len(new_test_in)

    print("\nFinished upsampling data!")
        
    # to numpy array    
    train_out = np.array(train_out)
    test_out = np.array(test_out)


    #shuffle data
    c = list(zip(train_in, train_out))
    random.shuffle(c)
    train_in, train_out = zip(*c)
    
    
    print("\nFinished shuffling training set")
    
#     # Split train, test
#     trainlen = int(0.6*data_size)
#     train_in, train_out = input[0:trainlen], output[0:trainlen]
#     test_in, test_out = input[trainlen:data_size], output[trainlen:data_size]
#     print("Finished split data!")
    
    
    
    data_dim = 1025 * window_size
    timesteps = None
    

    # expected input data shape: (batch_size, timesteps, data_dim)
    model = Sequential()
    model.add(LSTM(144, return_sequences=True, input_shape=(None, data_dim)))  # returns a sequence of vectors of dimension 32
    model.add(LSTM(144, return_sequences=True))  # returns a sequence of vectors of dimension 32
    model.add(LSTM(144))  # return a single vector of dimension 32
#     model.add(Reshape((12,12,1), input_shape=(144, )))
#     model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
#                  activation='relu',
#                  input_shape=(1,12,12,1)))
   
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Flatten())
    
    model.add(Dense(6))#fully connected
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    def accuracy(model, input, output):
        sumTrue = 0
        for x,y in zip(input, output):
            p = model.predict_classes(np.array([x]))
            if (p[0] == np.argmax(y)):
                sumTrue = sumTrue + 1
        
        return sumTrue / len(input)
            
    
    try:
        step = 0
        for i in range (0,4):
            for seq, label in zip(train_in, train_out):
                step = step + 1
                weight_sample = class_weight[np.argmax(label)]
                #sample_weight = np.array([weight_sample])

                model.train_on_batch(np.array([seq]), np.array([label]))
                if (step == 1 or step % 500 == 0):

                    print("Trained 500 sample!")
                    print("Score train set: ", accuracy(model, train_in, train_out))
                    print("Score test set; ", accuracy(model, test_in, test_out))
    except Exception as e:
        print("This is error: ", e)

    
    print("\nAfter training........................................................")
    print("Score train set: ", accuracy(model, train_in, train_out))
    print("Score test set; ", accuracy(model, test_in, test_out))
    

    ps = []
    for x, y in zip (test_in, test_out):
        p = model.predict_classes(np.array([x]))
#         print(p)
        ps.append(p)
    r = np.argmax(test_out, axis=1)
    #p = model.predict_classes(test_in)
   
    matrix = confusion_matrix(r, ps)
    print("\nConfusion matrix: \n", matrix)
    sum_colum = np.sum(matrix, axis = 0)
   # print("\nsum_column:", sum_colum)
    sum_row = np.sum(matrix, axis = 1)
    #print("\nsum_row:", sum_row)
    TP = [matrix[i,i] for i in range(0, len(matrix))]
    print("\nTP: ", TP,"\n")   
    FP = [sum_colum[i] - matrix[i,i] for i in range(0, len(matrix))]
    print("FP: ", FP,"\n")
    FN = [sum_row[i] - matrix[i,i] for i in range(0, len(matrix))]
    print("FN: ", FN,"\n")
    Presision = [TP[i] /(TP[i] + FP[i])  for i in range(0, len(matrix))]
    Recall = [TP[i] /(TP[i] + FN[i])  for i in range(0, len(matrix))]
    F1_score = [2 * Presision[i] * Recall[i] /(Presision[i] + Recall[i])  for i in range(0, len(matrix))]
    
  
    
    print("\nPrecision: ", Presision,"\n")
    print("Recall: ", Recall,"\n")
    print("F1_scrore: ", F1_score, "\n")
    
        
    #Your statements here

    stop = timeit.default_timer()

    print ("Time for training and testing: ", stop - start, "(s)") 

    